# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 28

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [8]:
#metric = 'valuePoints metric'
metric = 'adjusted points per game'
#metric = 'form 10'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
411,Patrício,Wolves,51,28.205128,110,124.101522,3.9,4.399963,5.010994,4.722631,1.948336
340,Lloris,Tottenham,53,10.975610,45,47.409139,4.1,4.319499,4.951467,5.155079,1.876271
93,Pope,Burnley,48,28.000000,126,116.118562,4.5,4.147091,5.592201,5.462394,1.892880
471,Henderson,Sheffield Utd,52,26.136364,115,108.064108,4.4,4.134627,4.898994,4.886258,1.813154
603,Reina,Aston Villa,45,5.000000,18,20.637434,3.6,4.127487,4.891413,4.642477,1.945716
131,Guaita,Crystal Palace,51,25.853659,106,101.292273,4.1,3.917909,4.611911,3.983277,1.734879
189,Alisson,Liverpool,62,19.777778,89,77.204007,4.5,3.903573,2.999412,4.043265,1.567713
168,Schmeichel,Leicester City,53,28.250000,113,110.207120,4.0,3.901137,5.894752,4.803272,1.694546
235,de Gea,Manchester Utd,54,27.647059,94,105.396259,3.4,3.812205,3.277745,3.144038,1.640512


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
182,Alexander-Arnold,Liverpool,78,27.796610,164,154.984838,5.9,5.575674,5.303373,5.310715,1.996412
103,Alonso,Chelsea,60,10.000000,71,51.902993,7.1,5.190299,8.763065,8.581485,2.118931
181,Robertson,Liverpool,70,27.959184,137,137.181573,4.9,4.906494,4.466164,4.583144,1.854480
401,Doherty,Wolves,62,25.813953,111,120.592389,4.3,4.671597,6.453053,5.435851,1.876159
297,Lundstram,Sheffield Utd,49,25.777778,116,114.617749,4.5,4.446378,2.592703,2.650007,2.008669
105,Azpilicueta,Chelsea,58,25.625000,82,112.926280,3.2,4.406879,3.704325,4.932848,1.829857
183,van Dijk,Liverpool,66,28.125000,135,118.183113,4.8,4.202066,3.490923,4.094036,1.635652
208,Otamendi,Manchester City,50,18.055556,65,73.916632,3.6,4.093844,4.161162,4.095888,1.830823
185,Matip,Liverpool,52,8.048780,33,32.128826,4.1,3.991763,2.830943,2.830943,1.750504


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,128,24.929577,177,169.449002,7.1,6.797107,5.147670,6.225873,1.899849
618,Fernandes,Manchester Utd,83,4.000000,32,25.629031,8.0,6.407258,8.052628,7.886019,2.223992
215,De Bruyne,Manchester City,107,26.176471,178,159.603829,6.8,6.097225,6.026669,5.668131,1.863976
192,Mané,Liverpool,124,24.923077,162,148.570653,6.5,5.961168,3.446638,4.150742,1.692858
214,Sterling,Manchester City,117,23.200000,116,128.563409,5.0,5.541526,4.882934,4.383279,1.620080
342,Son,Tottenham,98,21.034483,122,107.435485,5.8,5.107589,7.950907,6.492838,1.631559
217,Mahrez,Manchester City,84,21.818182,120,104.228153,5.5,4.777124,6.526757,5.298195,1.648264
150,Richarlison,Everton,83,26.200000,131,120.966576,5.0,4.617045,6.772790,6.647477,1.602600
344,Alli,Tottenham,83,20.980392,107,96.726570,5.1,4.610332,4.819716,4.776725,1.600270


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
233,Rashford,Manchester Utd,89,21.967213,134,125.600150,6.1,5.717619,NaN,6.186154,1.916550
166,Vardy,Leicester City,98,25.161290,156,136.032180,6.2,5.406407,3.014097,3.073232,1.727013
210,Agüero,Manchester City,119,20.847458,123,109.028120,5.9,5.229804,4.391398,5.278193,1.516044
409,Jiménez,Wolves,80,27.884615,145,143.812180,5.2,5.157402,4.986773,5.246503,1.823417
187,Firmino,Liverpool,97,27.826087,128,139.836241,4.6,5.025365,4.851852,5.405589,1.613547
11,Aubameyang,Arsenal,110,25.000000,150,124.796090,6.0,4.991844,5.699731,4.913020,1.505097
460,Abraham,Chelsea,75,25.192308,131,123.160150,5.2,4.888800,2.999551,3.972818,1.785137
278,Pukki,Norwich City,65,27.173913,125,129.116090,4.6,4.751472,3.414720,4.388894,1.863681
313,Ings,Southampton,72,28.163265,138,126.492030,4.9,4.491384,2.807573,3.412773,1.673840


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
metric = 'adjusted points per game'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,128,24.929577,177,169.449002,7.1,6.797107,5.147670,6.225873,1.899849
618,Fernandes,Manchester Utd,83,4.000000,32,25.629031,8.0,6.407258,8.052628,7.886019,2.223992
215,De Bruyne,Manchester City,107,26.176471,178,159.603829,6.8,6.097225,6.026669,5.668131,1.863976
192,Mané,Liverpool,124,24.923077,162,148.570653,6.5,5.961168,3.446638,4.150742,1.692858
233,Rashford,Manchester Utd,89,21.967213,134,125.600150,6.1,5.717619,NaN,6.186154,1.916550
182,Alexander-Arnold,Liverpool,78,27.796610,164,154.984838,5.9,5.575674,5.303373,5.310715,1.996412
214,Sterling,Manchester City,117,23.200000,116,128.563409,5.0,5.541526,4.882934,4.383279,1.620080
166,Vardy,Leicester City,98,25.161290,156,136.032180,6.2,5.406407,3.014097,3.073232,1.727013
210,Agüero,Manchester City,119,20.847458,123,109.028120,5.9,5.229804,4.391398,5.278193,1.516044


In [10]:
metric = 'form 10'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
103,Alonso,Chelsea,60,10.000000,71,51.902993,7.1,5.190299,8.763065,8.581485,2.118931
618,Fernandes,Manchester Utd,83,4.000000,32,25.629031,8.0,6.407258,8.052628,7.886019,2.223992
150,Richarlison,Everton,83,26.200000,131,120.966576,5.0,4.617045,6.772790,6.647477,1.602600
342,Son,Tottenham,98,21.034483,122,107.435485,5.8,5.107589,7.950907,6.492838,1.631559
191,Salah,Liverpool,128,24.929577,177,169.449002,7.1,6.797107,5.147670,6.225873,1.899849
233,Rashford,Manchester Utd,89,21.967213,134,125.600150,6.1,5.717619,NaN,6.186154,1.916550
410,Jota,Wolves,63,23.888889,86,96.476090,3.6,4.038534,7.522126,6.122215,1.608990
525,Sarr,Watford,63,18.095238,76,67.236330,4.2,3.715692,7.744234,6.056733,1.480367
147,Calvert-Lewin,Everton,64,25.952381,109,104.992030,4.2,4.045564,6.236504,5.687180,1.599150
